In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import yaml

class parse(object):
    def __init__(self, path):
        with open(path, 'r') as file:
            self.parameters = yaml.safe_load(file)

    def __getitem__(self, key):
        return self.parameters[key]

    def save(self, filename):
        with open(filename, 'w') as f:
            yaml.dump(self.parameters, f)

In [ ]:
class neuron:
    def __init__(self, init_params):
        self.time_steps = init_params['time_steps']
        self.membrane = np.zeros(time_steps)
        self.spike_train = np.zeros(time_steps)
        self.threshold = init_params['threshold']
        if init_params['func_type'] = 'exp':
            self.rho_0 = 1/np.exp(self.threshold)

class recurrent_network:
    def __init__(self, init_params):
        self.neurons_list = [neuron(init_params)]*10

0.36787944117144233